# Programación Lineal

Optimización Lineal, Forma Estándar, Dualidad y Resolución Computacional

## Resumen

Se desarrolla formalmente el modelo de programación lineal como técnica de optimización de funciones lineales sujetas a restricciones lineales. Se presenta su formulación matricial, la interpretación geométrica mediante politopos convexos, la transformación a forma estándar mediante variables de holgura y exceso, la forma distensionada para aplicación del algoritmo simplex y el análisis estructural de la dualidad (teorema débil y fuerte). Se incluyen implementaciones computacionales en Python para resolución geométrica en dos variables, conversión a forma estándar y evaluación del óptimo.


# 1. Planteamiento General

Considérese el problema general:

$$
\text{Minimizar } c_1 x_1 + c_2 x_2 + \dots + c_n x_n
$$

Sujeto a:

$$
a_{11}x_1 + a_{12}x_2 + \dots + a_{1n}x_n \ge b_1
$$
$$
a_{21}x_1 + a_{22}x_2 + \dots + a_{2n}x_n \ge b_2
$$
$$
\vdots
$$
$$
a_{m1}x_1 + a_{m2}x_2 + \dots + a_{mn}x_n \ge b_m
$$

Con:

$$
x_1,\dots,x_n \ge 0
$$

## Forma Matricial

$$
\text{Min } c^T x
$$

$$
Ax \ge b
$$

$$
x \ge 0
$$

La región factible queda definida como:

$$
F = \{ x \in \mathbb{R}^n : Ax \ge b, x \ge 0 \}
$$

Geométricamente, la región factible es un **politopo convexo** (intersección de semiespacios lineales).


# 2. Interpretación Geométrica

En dimensión 2:

* Cada restricción define un semiespacio.
* La intersección de todos ellos define la región factible.
* La solución óptima ocurre en un vértice del politopo.

# 3. Ejemplo de Maximización

## Problema de bicicletas

Variables:

$$
x = \text{bicicletas de montaña}
$$
$$
y = \text{bicicletas de paseo}
$$

Función objetivo:

$$
f(x,y) = 200x + 150y
$$

Restricciones:

$$
x + 2y \le 80
$$
$$
3x + 2y \le 120
$$
$$
x \ge 0, y \ge 0
$$

## Cálculo de vértices

Resolviendo:

$$
x+2y=80
$$
$$
3x+2y=120
$$

Se obtiene:

$$
(20,30)
$$

Otros vértices:

$$
(0,0),;(40,0),;(0,40)
$$

Evaluación:

$$
f(20,30)=8500
$$

Solución óptima:

$$
(20,30)
$$

# 4. Implementación Computacional en Python

## 4.1 Resolución por método de vértices (2 variables)

In [5]:
import numpy as np

def intersection(L1, L2):
    A = np.array([[L1[0], L1[1]],
                  [L2[0], L2[1]]], dtype=float)
    B = np.array([L1[2], L2[2]], dtype=float)
    if abs(np.linalg.det(A)) < 1e-12:
        return None
    return np.linalg.solve(A, B)

def feasible(pt, constraints):
    x, y = pt
    for a, b, c, s in constraints:
        val = a*x + b*y
        if s == "<=" and val > c + 1e-9:
            return False
        if s == ">=" and val < c - 1e-9:
            return False
    return True

def solve_lp_2d(obj, constraints, lines, mode="max"):
    pts = []
    for i in range(len(lines)):
        for j in range(i+1, len(lines)):
            p = intersection(lines[i], lines[j])
            if p is not None:
                pts.append(p)

    feas = [p for p in pts if feasible(p, constraints)]
    vals = [(obj[0]*p[0] + obj[1]*p[1], p) for p in feas]

    if mode == "max":
        best = max(vals, key=lambda x: x[0])
    else:
        best = min(vals, key=lambda x: x[0])

    return best, feas


constraints = [
    (1,2,80,"<="),
    (3,2,120,"<="),
    (1,0,0,">="),
    (0,1,0,">=")
]

lines = [
    (1,2,80),
    (3,2,120),
    (1,0,0),
    (0,1,0)
]

best, vertices = solve_lp_2d((200,150), constraints, lines, mode="max")

print("Vértices factibles:", vertices)
print("Óptimo:", best)

Vértices factibles: [array([20., 30.]), array([ 0., 40.]), array([40.,  0.]), array([0., 0.])]
Óptimo: (np.float64(8500.0), array([20., 30.]))


# 5. Forma Estándar

Un problema puede transformarse en:

$$
\text{Min } c^T x
$$

$$
Ax = b
$$

$$
x \ge 0
$$

Introduciendo:

* Variables de holgura si $Ax \le b$
* Variables de exceso si $Ax \ge b$

Si el problema es de maximización:

$$
\max c^T x = -\min(-c^T x)
$$

## Implementación de conversión a forma estándar

In [6]:
def to_standard_leq(A, b):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    m, n = A.shape
    I = np.eye(m)
    return np.hstack([A, I]), b

A = [[1,2],[3,2]]
b = [80,120]

A_std, b_std = to_standard_leq(A, b)
print("A estándar:\n", A_std)

A estándar:
 [[1. 2. 1. 0.]
 [3. 2. 0. 1.]]


# 6. Forma Distensionada

Para aplicar simplex:

$$
Ax + s = b
$$

$$
x \ge 0, s \ge 0
$$

Se trabaja con matriz aumentada:

$$
[A \mid I \mid b]
$$

# 7. Dualidad

## Forma Primal

$$
\max c^T x
$$
$$
Ax \le b
$$
$$
x \ge 0
$$

## Forma Dual

$$
\min b^T y
$$
$$
A^T y \ge c
$$
$$
y \ge 0
$$

## Teorema Débil

$$
c^T x \le b^T y
$$

## Teorema Fuerte

Si $x^*$ es óptimo primal y $y^*$ óptimo dual:

$$
c^T x^* = b^T y^*
$$

# 8. Ejemplo de Minimización (Vitaminas)

$$
\min C = 10y_1 + 12y_2
$$

Sujeto a:

$$
y_1+2y_2 \ge 120
$$
$$
2y_1+3y_2 \ge 180
$$
$$
3y_1+y_2 \ge 150
$$
$$
y_1,y_2 \ge 0
$$

In [7]:
constraints_v = [
    (1,2,120,">="),
    (2,3,180,">="),
    (3,1,150,">="),
    (1,0,0,">="),
    (0,1,0,">=")
]

lines_v = [
    (1,2,120),
    (2,3,180),
    (3,1,150),
    (1,0,0),
    (0,1,0)
]

best_v, vertices_v = solve_lp_2d((10,12), constraints_v, lines_v, mode="min")

print("Costo mínimo:", best_v)

Costo mínimo: (np.float64(864.0), array([36., 42.]))


Resultado esperado:

$$
C = 864
$$

# 9. Problemas Ilimitados o Inviables

* Si la región factible es vacía → problema inviable.
* Si la función crece sin límite → problema ilimitado.
* Si el primal es ilimitado → el dual es inviable.
* Ambos pueden ser inviables.

# 10. Conclusiones

La programación lineal constituye:

* Un modelo algebraico de optimización lineal.
* Una representación matricial formal $Ax \le b$.
* Una estructura geométrica convexa.
* Un sistema dual con igualdad estructural de óptimos.
* Una base para el método simplex y algoritmos de optimización.

El óptimo se encuentra en vértices extremos del politopo factible.
La forma estándar y distensionada permiten su tratamiento computacional sistemático.
La dualidad proporciona una interpretación económica y estructural del problema.